# Denpendency

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/Colab')

Mounted at /content/drive/


In [2]:
import re
import csv
import pandas as pd
import numpy as np

In [14]:
from sklearn.model_selection import train_test_split

In [4]:
# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 

!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

     |████████████████████████████████| 2.6 MB 6.5 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=ec064afefe3330b01caafb66eb1a15dd4e35814a9096482ce3509e40b3f3f3ac
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp
--2021-11-25 02:51:38--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  57.1MB/s    in 0.5s    

2021-11-25 02:51:38 (57.1 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2021-11-25 02:51:38--

In [5]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/drive/MyDrive/Colab Notebooks/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

text = "Đại học Quốc Gia Hà Nội"

word_segmented_text = rdrsegmenter.tokenize(text) 

In [6]:
word_segmented_text

[['Đại_học', 'Quốc_Gia', 'Hà_Nội']]

# Load and clean data

In [7]:
DATA_ROOT_DIR="/content/drive/MyDrive/Colab"

TRAIN_FILE = "/content/drive/MyDrive/Colab/data/train.crash"
TEST_FILE = "/content/drive/MyDrive/Colab/data/test.crash"

TRAIN_CSV = "/content/drive/MyDrive/Colab/data/train.csv"
TEST_CSV = "/content/drive/MyDrive/Colab/data/test.csv"

DEV_TRAIN_CSV = "/content/drive/MyDrive/Colab/data/train_dev.csv"
VAL_CSV = "/content/drive/MyDrive/Colab/data/test_dev.csv"

In [8]:
convert_word = {
    "ship": "vận chuyển",
    "shop": "cửa hàng",
    "m": "mình",
    "mik": "mình",
    "ko": "không",
    "k": " không ",
    "kh": "không",
    "khong": "không",
    "kg": "không",
    "khg": "không",
    "tl": "trả lời",
    "r": "rồi",
    "fb": "mạng xã hội",
    "face": "mạng xã hội",
    "thanks": "cảm ơn",
    "thank": "cảm ơn",
    "tks": "cảm ơn",
    "tk": "cảm ơn",
    "ok": "tốt",
    "dc": "được",
    "vs": "với",
    "đt": "điện thoại",
    "thjk": "thích",
    "thik":"thích",
    "qá": "quá",
    "trể": "trễ",
    "bgjo": "bao giờ",
    "h":"giờ",
    "qa":"quá",
    "dep":"đẹp",
    "xau":"xấu",
    "ib":"nhắn tin",
    "cute":"dễ thương",
    "sz":"size",
    "good":"tốt",
    "god":"tốt",
    "bt":"bình thường",
}


def tokmap(tok):
    if tok.lower() in convert_word:
        return convert_word[tok.lower()]
    else:
        return tok


def preprocess(review):
    review = review.lower()
    tokens = review.split()
    tokens = map(tokmap, tokens)
    return " ".join(tokens)


def load_data(filepath, is_train=True):
    regex = 'train_'
    if not is_train:
        regex = 'test_'

    item = []
    listItem = []

    with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if regex in line:
                listItem.append(item)
                item = [line]
            elif line != "\n" and line != "":
                item.append(line)

        listItem.append(item)
    listItem = listItem[1:]
    lst = []
    for e in listItem:
        idx = e[0]
        if is_train:
            label = int(e.pop(-1))
        review = " ".join(e[1:])
        review = re.sub(r"^\"*", "", review)
        review = re.sub(r"\"*$", "", review)
        review = preprocess(review)
        review_ws = rdrsegmenter.tokenize(review)
        review_ws = ' '.join([' '.join(x) for x in review_ws])
        if is_train:
            lst.append([idx, review, review_ws, label])
        else:
            lst.append([idx, review, review_ws])
    return lst

In [9]:
# Sau chi can dung ha
def load_csv_data(filepath, textcol="text"):
    df = pd.read_csv(filepath)
    samples = [ str(text) for text in df[textcol] ]
    labels  = [ intent for intent in df["label"] ]

    return samples, labels

In [10]:
train_data = load_data(TRAIN_FILE)
test_data = load_data(TEST_FILE, is_train=False)

print("# Loaded training samples: {}".format(len(train_data)))
print("# Loaded test samples: {}".format(len(test_data)))

# Loaded training samples: 16087
# Loaded test samples: 10981


In [11]:
cols = ["id", "text", "text_ws", "label"]
colsTest = ["id", "text", "text_ws"]
df_train = pd.DataFrame(data=train_data, columns=cols)
df_train.to_csv(TRAIN_CSV, index=False, quoting=csv.QUOTE_NONNUMERIC)
df_test = pd.DataFrame(data=test_data, columns=colsTest)
df_test.to_csv(TEST_CSV, index=False, quoting=csv.QUOTE_NONNUMERIC)

In [15]:
train_labels = [tp[-1] for tp in train_data]
dev_train_data, val_data, _, _ = train_test_split(train_data,
                                                  train_labels, test_size=0.2)
df_train_data = pd.DataFrame(data=dev_train_data, columns=cols)
df_val_data = pd.DataFrame(data=val_data, columns=cols)

df_train_data.to_csv(DEV_TRAIN_CSV, index=False, quoting=csv.QUOTE_NONNUMERIC)
df_val_data.to_csv(VAL_CSV, index=False, quoting=csv.QUOTE_NONNUMERIC)

In [16]:
df_train_data

,id,text,text_ws,label
0,train_015856,chất lượng sản phẩm tuyệt vời chất lượng sản p...,chất_lượng sản_phẩm tuyệt_vời chất_lượng sản_p...,0
1,train_001066,chất lượng kém ..đặt 5 áo giao 4 áo ..quá thất...,chất_lượng kém .. đặt 5 áo giao 4 áo .. quá th...,1
2,train_003188,chất lượng sản phẩm tuyệt vời bao bì tuyệt vời...,chất_lượng sản_phẩm tuyệt_vời bao_bì tuyệt_vời...,0
3,train_001218,tiki giao hàng nhanh sản phẩm chất lượng kém p...,tiki giao hàng nhanh sản_phẩm chất_lượng kém p...,1
4,train_012425,không như mong đợi bật 200°c với 20p mà đậu kh...,không như mong_đợi bật 200 °c với 20p mà đậu k...,1
...,...,...,...,...
12864,train_009360,đồng hồ đẹp phù hợp giá km chất lượng thỳ phải...,đồng_hồ đẹp phù_hợp giá km chất_lượng thỳ phải...,0
12865,train_006161,chất lượng kém. đúng tiền nào của nấy. xin chừ...,chất_lượng kém . đúng tiền_nào_của_nấy . xin c...,1
12866,train_010163,dang sd chưa biết đóng gói sản phẩm rất đẹp và...,dang sd chưa biết đóng_gói sản_phẩm rất đẹp và...,0
12867,train_013520,cửa hàng chăm sóc khách hàng nhiệt tình. chất ...,cửa_hàng chăm_sóc khách_hàng nhiệt_tình . chất...,0


In [17]:
df_val_data

,id,text,text_ws,label
0,train_010069,sản phâtm tốt! nhưng vì đặt nhầm! miếng dán gu...,sản phâtm tốt ! nhưng vì đặt nhầm ! miếng dán ...,0
1,train_015762,cặp sách bám bụi rất nhiều nhìn như đồ second ...,cặp sách bám bụi rất nhiều nhìn như đồ second ...,1
2,train_006633,thời gian giao hàng rất chậm. gần 2 tuần lễ. đ...,thời_gian giao hàng rất chậm . gần 2 tuần_lễ ....,1
3,train_004954,không ưng 1 chút nào,không ưng 1 chút nào,1
4,train_007483,chất lượng sản phẩm rất kém. chuôi cắm không ...,chất_lượng sản_phẩm rất kém . chuôi cắm không ...,1
...,...,...,...,...
3213,train_001512,dau num ti y ho to qua,dau num ti y ho to qua,1
3214,train_010415,cửa hàng gửi thiếu mình sản phẩm này nhưng đã ...,cửa_hàng gửi thiếu mình sản_phẩm này nhưng đã ...,0
3215,train_015645,chất lượng sản phẩm tuyệt vời đóng gói sản phẩ...,chất_lượng sản_phẩm tuyệt_vời đóng_gói sản_phẩ...,0
3216,train_008092,rửa mặt rất sạch. máy chạy êm. rất đáng tiền,rửa mặt rất sạch . máy chạy êm . rất đáng tiền,0
